In [8]:
import pandas as pd
import httpx
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

In [2]:
DATA_PATH = "./data/alt-cleaned_resume.csv"
FAISS_PATH = "./alt-vectorstore"
RAG_K_THRESHOLD = 2
EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
LLM_MODEL = "gpt-35-turbo"
OPENAI_ENDPOINT = "https://aalto-openai-apigw.azure-api.net"
OPENAI_KEY = ""

In [4]:
def update_base_url(request: httpx.Request):
  if request.url.path == "/chat/completions":
    request.url = request.url.copy_with(path="/v1/chat")

llm = ChatOpenAI(
    default_headers={"Ocp-Apim-Subscription-Key": OPENAI_KEY},
    base_url="https://aalto-openai-apigw.azure-api.net",
    api_key=False,
    http_client=httpx.Client(
      event_hooks={
        "request": [update_base_url],
    }),
  )

In [6]:
df = pd.read_csv("../data/main-data/data-checkpoints/jd1.csv")
job_title_list = list(df["Job Title"])
job_description_list = list(df["Job Description"])
generated_resumes_list = []

In [7]:
for job_title, job_description in zip(job_title_list, job_description_list):
  user_message = HumanMessage(content="""
    You are an Al assistant that helps create resumes for a given job description.
    Generate 2 resumes for each job description, such that 1 resume is an almost perfect match, while the other resume is only slightly suitable. 
    The length of a resume should only be around 400 to 500 words. 
    Only keep details that are relevant in a resume and with minimal formatting.
  """)
  
  system_message = SystemMessage(content=f"""
    Here is the Job Description (Note that all required skills may not be present in resume and some nonrelevant skills/experience can be present).
    {job_title}, {job_description}
  """)

  response = llm([user_message, system_message])
  generated_resumes_list.append(response)

["Require iPhone App Developers having 3 to 7 years of hands-on programming experience.\nWork with one of the finest mobile apps production companies in Ahmedabad, India.\nPhoto & Video. Business. Finance. Productivity. Kids. All major app categories.\nWork where #quality is more valued than just delivering as per the scope.\nWork with cross-functional collaborative teams Analysts, Designers, and Architects!\nSwift. Core Data. Core Animation. Firebase. Debugging. Apple HIG. PN. APIs. GitLab. And CI. Deep understanding of UX.\nSuper-communicators: written and oral. (Non-negotiable).\nDynamic CTC. Earn 50% more if you can beat the expectations.\nFlexible work timings. Available as a perk to those who are attentive and exceed expectations.\nNo red-tape. Either you finish whats important, or you dont. Straightness. Applied.\nJob Type: Full-time\nSchedule:\nMorning shift\nExperience:\nwork: 1 year (Preferred)\ntotal work: 1 year (Preferred)\niOS: 1 year (Preferred)\nEducation:\nBachelor's (